In [1]:
import base64
import requests
import openai
import os
import json

In [2]:
openai.api_type = "azure"
openai.api_key = '252c6c79a5da4a80a61e54d315ba1fed'
openai.api_base = 'https://api.umgpt.umich.edu/azure-openai-api/ptu'
openai.api_version = '2023-03-15-preview'

In [4]:
for folder in os.listdir(os.getcwd() + "\\videos\\"):
    for folder2 in os.listdir(os.getcwd() + "\\vid_summary" + "\\" + folder.split(".")[0] + "_chunks"):
        try:
            os.mkdir(os.getcwd() + "\\summary\\" + folder.split(".")[0] + "\\" + folder2)
        except:
            pass

In [3]:
def combine(VIDEO, TRANSCRIPTION):
    response = openai.ChatCompletion.create(
            engine='gpt-4',
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Can you combine the following video description and video transcript to describe what is happening in the scene: Video description: {VIDEO} Video transcription: {TRANSCRIPTION}"}
            ]
        )
    return response


In [6]:
folders = os.listdir(os.getcwd() + "\\vid_summary\\")
for folder1 in folders:
    for folder2 in os.listdir(os.getcwd() + "\\vid_summary\\" + folder1):
        for file in os.listdir(os.getcwd() + "\\vid_summary\\" + folder1 + "\\" + folder2):
            VIDEO = open(os.getcwd() + "\\vid_summary\\" + folder1 + "\\" + folder2 + "\\" + file, "r").read()
            TRANSCRIPTION = ""
            with open(os.getcwd() + "\\video_chunks\\" + folder1 + "\\" + "full_transcript.json") as file:
                t = json.load(file)
                for key in t:
                    TRANSCRIPTION += t[key] + "\n"
            
            with open(os.getcwd() + "\\summary\\" + folder1.split("_")[0] + "\\" + folder2 + ".json", "w") as file:
                file.write(combine(VIDEO, TRANSCRIPTION)['choices'][0]['message']['content'])
            

In [4]:
def generate_questions(summary):
    response = openai.ChatCompletion.create(
            engine='gpt-4',
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Given the summary of a memory, generate an answerable question that is meant to jog the recorders memory. Summary {summary}"}
            ]
        )
    return response['choices'][0]['message']['content']

In [5]:
def generate_answer(summary, question):
    response = openai.ChatCompletion.create(
            engine='gpt-4',
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Given the summary of a memory and a question, find the answer. Summary: {summary} Question: {question}"}
            ]
        )
    return response['choices'][0]['message']['content']

In [9]:
qas = []
for folder1 in os.listdir(os.getcwd() + "\\summary\\"):
    for file in os.listdir(os.getcwd() + "\\summary\\" + folder1.split("_")[0]):
        with open(os.getcwd() + "\\summary\\" + folder1.split("_")[0] + "\\" + file, "r") as read:
            summary = read.read()
            question = generate_questions(summary)
            answer = generate_answer(summary, question)
            qas.append({
                'question': question,
                'video': "video_chunks\\" + folder1 + "_chunks\\" + file.split(".")[0] + ".mp4",
                'answer': answer
            })

with open("qas.json", "w") as file:
    json.dump(qas, file)